# Hyperparameter Search using TensorBoard to track metrics

### imports

In [1]:
import sys

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorboard.plugins.hparams import api as hp
from PIL import Image
from collections.abc import Mapping

sys.path.append('../')
from models import models


### Generators for loading and augmenting images

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   rotation_range=5,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/train',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../data/test',
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical')
                                            

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


### Setup for experiment

In [ ]:
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'RMSprop', 'sgd']))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.01, 0.005, 0.001]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_DROPOUT, HP_OPTIMIZER, HP_LEARNING_RATE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

### Training function

In [8]:
def train_test_model(hparams, tf_callback):
    model = models.XrayModelEfficient(hparams[HP_DROPOUT])

    optimizer = hparams[HP_OPTIMIZER]
    if optimizer == "adam":
        optimizer = tf.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE])
    elif optimizer == "sgd":
        optimizer = tf.optimizers.SGD(learning_rate=hparams[HP_LEARNING_RATE])
    elif optimizer=='RMSprop':
        optimizer = tf.optimizers.RMSprop(learning_rate=hparams[HP_LEARNING_RATE])

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer=optimizer,
                  metrics='accuracy')

    model.fit(train_generator,
              steps_per_epoch=100,
              epochs=3,
              callbacks=[tf_callback])
    
    _, accuracy = model.evaluate(test_generator)
    return accuracy

### Clear logs

In [9]:
%rm -rf ./logs/

### Run experiments

In [10]:
def run(run_dir, hparams, tf_callback):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams, tf_callback)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0

log_dir = "logs/fit/" + "run-%d" % session_num
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

for dp_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
  for lr_rate in HP_LEARNING_RATE.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
        HP_DROPOUT: dp_rate,
        HP_LEARNING_RATE: lr_rate,
        HP_OPTIMIZER: optimizer
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams, tensorboard_callback)
      session_num += 1

## Launch tensorboard

In [12]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
